In [ ]:
import numpy as np
from scipy import signal
from scipy.interpolate import interp1d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5*fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
def butter_passband_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5*fs
    normal_lowcut = lowcut / nyq
    normal_highcut = highcut / nyq
    b, a = signal.butter(order, [normal_lowcut, normal_highcut], btype='band', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
def logEnergy(sig,fs):
    logE=-1e30
    if len(sig)>0:
        sig2=np.power(sig,2)
        sumsig2 = np.sum(sig2)/len(sig2)
        logE=10*np.log10(sumsig2)
    return logE

In [ ]:
def mu_ener_var(sig):
    feat_mu = np.mean(sig)
    feat = np.power(sig,2)
    feat_energy = np.sum(np.absolute(feat))
    feat_variance = np.var(sig)
    return np.hstack([feat_mu, feat_energy, feat_variance])

In [ ]:
def diffe(sig):
    sig = np.delete(sig, [14,15,18,23], axis=0)
    diferencias = []
    for i in range(int(len(sig)/2)):
        diferencias.append(sig[i]-sig[i+14])
    return np.vstack(diferencias)

In [ ]:
def extract_features(sigs, fs):
    GSR_ = GSR(sigs[36], fs)
    BVP_ = BVP(sigs[38], fs)
    Respiration_ = Respiration(sigs[37],fs)
    Temperature_ = Temperature(sigs[39],fs)
    EMGEOG_ = EMGEOG(sigs[32],sigs[33],sigs[34],sigs[35],fs)
    EEG_ = EEG(sigs[0:32],fs)
    return np.hstack([GSR_, BVP_, Respiration_, Temperature_, EMGEOG_, EEG_])

In [ ]:
def EEG(feats,fs):
    dife = diffe(feats)
    order=2
    power = []
    
    feats = np.vstack([feats,dife])
    
    for i in range(feats.shape[0]):
        sig = feats[i]
        for x in range(4):
            if x == 0:
                lowcut = 3.5
                highcut = 7.5
            if x == 1:
                lowcut = 8
                highcut = 13
            if x == 2:
                lowcut = 12
                highcut = 30
            if x == 3:
                lowcut = 25
                highcut = 60
            
            sig_filt = butter_passband_filter(sig, lowcut, highcut, fs, order)
            #f, Pxx = signal.welch(sig_filt, fs, nperseg=64, window=np.ones(64))
            #AbsPxx = np.sum(Pxx)
            
            size_frameS=0.02*float(fs)
            size_stepS=0.01*float(fs)
            overlap=size_stepS/size_frameS
            nF=int((len(sig_filt)/size_frameS/overlap))-1
            
            power_first = []
            for l in range(nF):
                feat_frame = sig_filt[int(l*size_stepS):int(l*size_stepS+size_frameS)]
                power_f = logEnergy(feat_frame,fs)
                power_f = np.power(10,(power_f/10))
                power_first.append(power_f)
                
            power.append(np.mean(power_first))
            #LogPxx = 20*np.log(AbsPxx)
            
    
    #alpha 8-13Hz
    #beta 12-30Hz
    #theta 3.5-7.5Hz
    #gamma 25-100Hz
    
    
    return np.hstack(power)

In [ ]:
def EMGEOG(feat1, feat2, feat3, feat4, fs):
    
    feat_one = mu_ener_var(feat1)
    feat_two = mu_ener_var(feat2)
    feat_three = mu_ener_var(feat3)
    feat_four = mu_ener_var(feat4)
    
    return np.hstack([feat_one, feat_two, feat_three, feat_four])

In [ ]:
def Temperature(feat,fs):
    
    feat_mu = np.mean(feat)
    feat_diff = np.mean(np.diff(feat))
    
    order=2
        
    cutoff = 0.1
    lowcut = 0.1
    highcut = 0.2
    
    size_frameS=0.02*float(fs)
    size_stepS=0.01*float(fs)
    overlap=size_stepS/size_frameS
    nF=int((len(feat)/size_frameS/overlap))-1
    power_first = []
    power_second = []
    
    first_feat = butter_lowpass_filter(feat, cutoff, fs, order)
    second_feat = butter_passband_filter(feat, lowcut, highcut, fs, order)
    
    for l in range(nF):
        feat_frame = first_feat[int(l*size_stepS):int(l*size_stepS+size_frameS)]
        power_f = logEnergy(feat_frame,fs)
        power_f = np.power(10,power_f/10)
        power_first.append(power_f)
        
        feat_frame = second_feat[int(l*size_stepS):int(l*size_stepS+size_frameS)]
        power_s = logEnergy(feat_frame,fs)
        power_s = np.power(10,power_s/10)
        power_second.append(power_s)
        
        
    power_first_mu = np.mean(power_first)
    power_second_mu = np.mean(power_second)
    
    return np.hstack([feat_mu, feat_diff, power_first_mu, power_second_mu])

In [ ]:
def Respiration(feat,fs):
    
    order=2
        
    #first passband filter
    lowcut = 0.05
    highcut = 0.25
        
    first_feat = butter_passband_filter(feat, lowcut, highcut, fs, order)
        
    #second passband filter
    lowcut = 0.25
    highcut = 0.5
        
    second_feat = butter_passband_filter(feat, lowcut, highcut, fs, order)
    
    energy_feat_first = logEnergy(first_feat,fs)
    energy_feat_second = logEnergy(second_feat,fs)
    
    energy_rate = energy_feat_first-energy_feat_second
    
    feat_mu = np.mean(feat)
    
    feat_diff_mu = np.mean(np.diff(feat))
    
    feat_max = np.max(feat)
    
    return np.hstack([energy_rate, feat_mu, feat_diff_mu, feat_max])

In [ ]:
def BVP(feat, fs):
    
    size_frameS=10*float(fs)
    size_stepS=5*float(fs)
    overlap=size_stepS/size_frameS
    nF=int((len(feat)/size_frameS/overlap))-1

    hrs=[]
    hrvs=[]
    rrs_f=[]

    for l in range(nF):
        feat_frame = feat[int(l*size_stepS):int(l*size_stepS+size_frameS)]
        
        feat_frame = 10*np.diff(feat_frame,1)
        
        peaks, _ = signal.find_peaks(feat_frame, prominence = 0.03)
        peaks_copy = peaks
        
        feat_copy = feat_frame[peaks]
        
        peaks1, _ = signal.find_peaks(np.concatenate(([0],feat_frame[peaks],[0])), prominence = 0.03)
        
        feat_copy1 = feat_copy[peaks1-1]
        
        peaks2 = []
        for i in range(len(feat_copy1)):
            idx= np.where(feat_copy1[i]==feat_copy)[0][0]
            peaks2.append(peaks_copy[idx])
            
            feat_copy = np.delete(feat_copy, idx)
            peaks_copy = np.delete(peaks_copy, idx)
            
        peaks2= np.hstack(peaks2)
        
       
        
        rrs = []
        rmssd = []
        cont = 0
        for i in range(len(peaks2)-2):
            one = peaks2[i]
            two = peaks2[i+1]
            interval = (two - one)/fs
            rrs.append(interval)
        rrs_copy = rrs.copy()
        rrs.reverse()
        rmssd = np.diff(rrs)
        rmssd = np.power(rmssd,2)
        rmssd = rmssd[::-1]
        
            # cont = cont + 1
            # if i != 0:
            #     pw = np.power((rrs[i-1]-rrs[i]),2)
            #     rmssd.append(pw)
            #     cont = 1
        
        rmssd = np.power(np.sum(rmssd),1/2)
        hr = len(peaks2)/(size_frameS/fs)
        
        hrs.append(hr)
        hrvs.append(rmssd)
        rrs_f.append(rrs_copy)
        
   
    hrs = static_feats(hrs)
    hrvs = static_feats(hrvs)
    rrs_f = static_feats(np.hstack(rrs_f))    

    
    return np.hstack([hrs, hrvs, rrs_f])

In [ ]:
def GSR(feat, fs):
    
    dfeat = np.diff(feat, 1, axis=0)
    
    #Average skin resistance, average of derivative, average of derivative for negative values only
    feat_mu = np.mean(feat,0)
    dfeat_mu = np.mean(dfeat,0)
    ndfeat_mu = np.mean(np.where(dfeat < 0)[0],0)
    
    #Filter parameters
    cutoff = 2.4
    order=2
    
    #filter
    feat = butter_lowpass_filter(feat, cutoff, fs, order)
    
    #spectral power
    fourier_transform = np.fft.rfft(feat)
    abs_fourier_transform = np.abs(fourier_transform)
    power_spectrum = np.square(abs_fourier_transform)
    frequency = np.linspace(0, 128/2, len(power_spectrum))
    #    plt.plot(frequency, abs_fourier_transform)
    
    #select the first ten spectral power
    peaks, _ = signal.find_peaks(power_spectrum, height=0)
    

    
    return np.hstack([feat_mu, dfeat_mu, ndfeat_mu])

In [ ]:
def static_feats(featmat):
    """Compute static features
    :param featmat: Feature matrix
    :returns: statmat: Static feature matrix
    """
    mu = np.mean(featmat,0)
    st = np.std(featmat,0)
    statmat = np.hstack([mu,st])    
    return statmat